In [1]:
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pickle

##### Leer Dataset de Entrenamiento

In [2]:
# Leer el dataset
df = pd.read_csv("datasets/MasterBI_Train.csv", header=0, sep=",", encoding="utf-8")
# Crear una serie con la variable target
y = df['HasDetections']

# Mostrar todas las columnas del dataframe
pd.set_option('display.max_columns',None)
df.head()


C:\Users\martin.salas\AppData\Local\Temp\ipykernel_37112\246698288.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/MasterBI_Train.csv", header=0, sep=",", encoding="utf-8")


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,7bd132f88d90f8c628ae7204ce1e0038,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1486.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,51,103807.0,27.0,98.0,103,windows10,x64,10.0.0.0,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NaN,0.0,117.0,ExistsNotSet,1.0,1.0,Notebook,Windows.Desktop,2668.0,171473.0,4.0,5.0,2697.0,NaN,953869.0,HDD,953253.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,NaN,0.000000e+00,10.0.16299.547,amd64,rs3_release_svc_escrow,16299,547,Core,CORE,IBSClean,5.0,26,UNKNOWN,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,628.0,12300.0,1,NaN,0.0,0,0,0.0,0.0,6.0,1
1,12678cb674ee9f2c85e526dc9774845d,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1234.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,35,27424.0,27.0,45.0,83,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,525.0,331262.0,2.0,5.0,1985.0,NaN,476940.0,HDD,190774.0,0,4096.0,Notebook,15.5,1366.0,768.0,Mobile,NaN,0.000000e+00,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,Reset,13.0,48,FullAuto,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,142.0,70187.0,0,NaN,0.0,0,0,0.0,1.0,4.0,0
2,624400f847781fe7ccc3546e921fb46f,win8defender,1.1.15000.2,4.18.1806.18062,1.271.545.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,211,10989.0,18.0,277.0,75,windows10,x64,10.0.0.0,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NaN,0.0,117.0,ExistsNotSet,1.0,1.0,Notebook,Windows.Desktop,4730.0,312820.0,8.0,5.0,2880.0,NaN,953869.0,HDD,940100.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,li-i,0.000000e+00,10.0.16299.492,amd64,rs3_release_svc_escrow,16299,492,Core,CORE,UUPUpgrade,8.0,31,Notify,0,IS_GENUINE,Retail,0.0,0.0,Retail,0.0,556.0,13909.0,1,0.0,0.0,0,0,0.0,0.0,10.0,1
3,5dd6c5c23e09214f43fbe89888bf98f7,win8defender,1.1.15100.1,4.11.15063.0,1.273.912.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,44,NaN,27.0,57.0,218,windows10,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,0,NaN,0.0,105.0,NaN,1.0,1.0,Desktop,Windows.Desktop,1980.0,228125.0,8.0,5.0,2970.0,NaN,476940.0,HDD,476438.0,0,8192.0,Desktop,23.1,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.15063.0,amd64,rs2_release,15063,0,CoreSingleLanguage,CORE

In [3]:
# Definir las variables identifier y target
identifier = 'MachineIdentifier' #no aporta valor al modelo
target = 'HasDetections' #variable objetivo

# Eliminar las columnas 'HasDetections' y 'MachineIdentifier' del dataframe
df.drop('HasDetections', axis=1, inplace=True)
df.drop('MachineIdentifier', axis=1, inplace=True)

In [4]:
# Llenar los Nulls de la variable 'Census_IsWIMBootEnabled' con 1 
df['Census_IsWIMBootEnabled'] = df['Census_IsWIMBootEnabled'].fillna(1)

# Llenar los Nulls de la variable 'PuaMode' con 1 
df['PuaMode'] = df['PuaMode'].fillna('off')
df['PuaMode'] = df['PuaMode'].replace({'off': 0, 'on': 1})

C:\Users\martin.salas\AppData\Local\Temp\ipykernel_37112\2029929336.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PuaMode'] = df['PuaMode'].replace({'off': 0, 'on': 1})


##### Eliminar variables con Alta Cardinalidad y con demasiados valores Nulos

In [5]:
# 1. Variables con alta cardinalidad
high_cardinality_threshold = 0.10  # 10% del total de filas
high_cardinality = []
for col in df.columns:
    n_unique = df[col].nunique(dropna=False)
    if n_unique / df.shape[0] > high_cardinality_threshold:
        high_cardinality.append(col)

# 2. Variables con demasiados valores desconocidos
missing_values_threshold = 0.60  # 30%
missing_values = []
for col in df.columns:
    if df[col].isna().sum() / df.shape[0] > missing_values_threshold:
        missing_values.append(col)

# Eliminar columnas con Alta cardinalidad
df.drop(high_cardinality, axis=1, inplace=True)

# Eliminar columnas con demasiados valores desconocidos
df.drop(missing_values, axis=1, inplace=True)

features = [col for col in df.columns if col not in [identifier, target]]

##### Clasificar las variables en Categoricas, Númericas y Booleanas

In [6]:
# Clasificación de variables
def clasificar_variables(df, cols):
    binary_cols, cat_cols, num_cols = [], [], []
    for col in cols:
        n_classes = df[col].nunique()
        if n_classes <= 2:
            binary_cols.append(col)
        elif df.shape[0] * 0.01 > n_classes > 2:
            cat_cols.append(col)
        else:
            num_cols.append(col)
    return binary_cols, cat_cols, num_cols

# Clasificar las variables
binary_cols, cat_cols, num_cols = clasificar_variables(df, features)

# Guardar en un diccionario las variables clasificadas
clasificacion_vars = {
    'binary_cols': binary_cols,
    'cat_cols': cat_cols,
    'num_cols': num_cols
}

# Guardar en archivo pickle el diccionario de variables clasificadas
with open("files/clasificacion_vars.pkl", "wb") as f:
    pickle.dump(clasificacion_vars, f)



##### Imputar valores nulos en dependencia al tipo de variable. Luego guardar los datos que se imputaron para cada columna.

In [7]:
# Diccionario para guardar los valores de imputación
valores_imputacion = {
    'binarias': {},
    'categoricas': {},
    'numericas': {}
}

# Imputación de variables binarias
def imputar_binarias(df, cols):
    for col in cols:
        if df[col].isnull().sum() == 0:
            continue
        value_counts = df[col].value_counts(normalize=True)
        if len(value_counts) == 0 or value_counts.max() > 0.8:
            df[col] = df[col].fillna(-1)
            valores_imputacion['binarias'][col] = -1
        elif abs(value_counts.get(0, 0) - value_counts.get(1, 0)) < 0.1:
            moda = df[col].mode()[0]
            df[col] = df[col].fillna(moda)
            valores_imputacion['binarias'][col] = moda
        else:
            probs = value_counts.to_dict()
            df[col] = df[col].apply(lambda x: np.random.choice([0, 1], p=[probs.get(0, 0.5), probs.get(1, 0.5)]) if pd.isnull(x) else x)
            valores_imputacion['binarias'][col] = probs
    return df

# Imputación de variables categóricas
def imputar_categoricas(df, cols):
    for col in cols:
        if df[col].isnull().sum() == 0:
            continue
        moda = df[col].mode()[0]
        freq = df[col].value_counts(normalize=True).get(moda, 0)
        valor = "missing" if freq > 0.8 else moda
        df[col] = df[col].fillna(valor)
        valores_imputacion['categoricas'][col] = valor
    return df

# Imputación de variables numéricas
def imputar_numericas(df, cols):
    for col in cols:
        if df[col].dtype == 'object':       
            try:
                df[col] = df[col].astype('float64')
            except ValueError:
                continue

        if df[col].isnull().sum() == 0:
            continue
        skew = df[col].dropna().skew()
        valor = df[col].mean() if abs(skew) < 0.5 else df[col].median()
        df[col] = df[col].fillna(valor)
        valores_imputacion['numericas'][col] = valor
    return df

# Aplicar imputaciones
df = imputar_binarias(df, binary_cols)
df = imputar_categoricas(df, cat_cols)
df = imputar_numericas(df, num_cols)

# Guardar el diccionario con lo valores de imputación por columna
with open("files/valores_imputacion.pkl", "wb") as f:
    pickle.dump(valores_imputacion, f)


##### Aplicar Target Encoding a las variables categoricas y guardar el encoder

In [8]:
# Aplicar el Target Encoding
X = df.copy()
te = TargetEncoder(cols = cat_cols, smoothing = 10)
te.fit(X[cat_cols], y)
X_te= X.join(te.transform(X[cat_cols]).add_suffix('_target'))
X_te = X_te.drop(cat_cols, axis=1)

# Guardar el TargetEncoder en un archivo
with open("files/target_encoder.pkl", "wb") as archivo:
    pickle.dump(te, archivo)



In [9]:
X = X_te.copy()
# Listar el nuevo nombre de las variables
features = [col for col in X.columns if col not in [identifier, target]]

In [10]:
# Dividir el dataset en entrenamiento y prueba (20% prueba, 80% entrenamiento)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size = 0.2, random_state = 42)

##### Entrenar un modelo inicial para determinar las variables con mayor importancia a la hora de predecir

In [11]:
# Entrenar un modelo inicial con el clasificador XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [12]:
# Realizar predicciones en el conjunto de prueba
xgb_predict = xgb_model.predict(X_test)

In [13]:
# Crear un DataFrame para la importancia de las variables
xgb_features = pd.DataFrame(features)
# Importancia de las variables según el modelo XGBoost
xgb_features['score'] = xgb_model.feature_importances_
xgb_features.columns = ['features','feature_score']
# Ordenar las variables por importancia
xgb_features.sort_values(by=['feature_score'],ascending = False,inplace = True)
# Mostrar las 20 variables más importantes
xgb_features.head(20)

,features,feature_score
42,SmartScreen_target,0.293626
26,AVProductStatesIdentifier_target,0.072333
16,Census_IsVirtualDevice,0.056108
58,Census_InternalBatteryNumberOfCharges_target,0.025207
49,Census_ProcessorModelIdentifier_target,0.025103
20,Wdft_IsGamer,0.024497
24,AvSigVersion_target,0.021788
27,AVProductsInstalled_target,0.018894
23,AppVersion_target,0.015292
10,Census_HasOpticalDiskDrive,0.015149


In [14]:
# Seleccionar/Filtrar las variables con importancia mayor a 0.01
xgb_features_importants = xgb_features[xgb_features['feature_score'] > 0.01]
# Guardar la lista de variables más importantes
final_features = xgb_features_importants.features.to_list()

# Guardar en archivo pickle la lista de variables finales a usar
with open("files/final_features.pkl", "wb") as f:
    pickle.dump(final_features, f)

##### Guardar los nuevos datos de entrenamiento para usarlos en las siguientes etapas

In [15]:
X = X.loc[:, final_features]
y = y.copy()

datos_entrenamiento = {
    'X': X,
    'y': y
}

# Guardar en archivo pickle
with open("files/datos_entrenamiento.pkl", "wb") as f:
    pickle.dump(datos_entrenamiento, f)